In [3]:
import numpy as np
from numpy.linalg import inv
from sklearn.datasets.samples_generator import make_blobs, make_classification
from sklearn.linear_model import LogisticRegression

# Data

In [58]:
sigmoid = lambda z: 1/(1+np.exp(-z))
N = 200
p = 1
# X, y = make_classification(n_samples=N, n_features=p, n_informative=p, n_redundant=0, n_repeated=0, n_clusters_per_class=1)
f = lambda x: 1.1 + 0.5*x
x = np.random.uniform(low=-5, high=5, size=N).reshape((-1,1))
y = np.random.binomial(n=1, p=sigmoid(f(x)).squeeze(), size=N)

## Logistic Regression

Logistic regression aims to model a binary outcome using a linear function in feature space. In particular, we model the log-odds as a linear combination of the predictor variables: $$\log{\frac{\mathrm{Pr}(y_i = 1|X)}{1-\mathrm{Pr}(y_i = 1|X)}} = \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} + \ldots + \beta_p x_{ip} = \mathbf{x}_i^T \boldsymbol{\beta}$$, where $\mathbf{x}_i=  (1, x_{i1}, \ldots, x_{ip})^T$.
One can show that this can be rewritten as: $$\mathrm{Pr}(y_i = 1|X)=\frac{\exp{\mathbf{x}_i^T \boldsymbol{\beta}}}{1+\exp{\mathbf{x}_i^T \boldsymbol{\beta}}}=\frac{1}{1+\exp{(-\mathbf{x}_i^T \boldsymbol{\beta})}}$$.

Assuming that $y \sim Bin(1,\mathrm{Pr}(y_i = 1|X))$ we can use the binomial likelihood or binomial cross-entropy loss to fit the model: $$L(\boldsymbol{\beta};\mathbf{X}, \mathbf{y}) = \prod_i^n p^{y_i} (1-p)^{1-y_i}$$ where $\mathbf{X} = (\mathbf{x}_1, \ldots, \mathbf{x}_n)^T$ is a design matrix with the feature vectors as rows, $\mathbf{y}=(y_1,\ldots,y_n)^T$, and $p=\mathrm{Pr}(y_i = 1|X)$. We can obtain the log-likelihood as $$\ell(\boldsymbol{\beta};\mathbf{X}, \mathbf{y})=\sum_i^n y_i \log{(p)} + (1-y_i)\log{(1-p)}$$.

*Note:*
* $\partial Ax / \partial x = A^T$
* $\partial x^TA / \partial x = A$

We now take the derivative with respect to $\boldsymbol{\beta}$ and set to zero. Note that $\frac{\partial p}{\partial \boldsymbol{\beta}} = p (1-p) \mathbf{x}_i$. We obtain $$\frac{\partial \ell}{\partial \boldsymbol{\beta}} = \sum_i^n \mathbf{x}_i^T (y_i - p_i)$$ (it takes a couple of lines to work through). We further get the second derivative as $$\frac{\partial \ell}{\partial \boldsymbol{\beta}\boldsymbol{\beta}^T} = \sum_i^n  p_i(1-p_i)\mathbf{x}_i\mathbf{x}_i^T$$. Now we can use Newton-Raphson to iteratively update the parameters: $$\boldsymbol{\beta}^{t+1} = \boldsymbol{\beta}^{t} - \left( \frac{\partial \ell}{\partial \boldsymbol{\beta}\boldsymbol{\beta}^T} \right)^{-1} \frac{\partial \ell}{\partial \boldsymbol{\beta}}$$ where the derivatives are evaluated at $\boldsymbol{\beta}^{t}$.

We can further write this as follows: $$\frac{\partial \ell}{\partial \boldsymbol{\beta}} = \mathbf{X}^T(\mathbf{y}-\mathbf{p})$$,
$$\frac{\partial \ell}{\partial \boldsymbol{\beta}\boldsymbol{\beta}^T}= -\mathbf{X}^T \mathbf{W} \mathbf{X}$$ where $\mathbf{W} = \mathrm{diag}(\mathbf{p}(1-\mathbf{p}))$ that is a diagonal matrix with $p_i(1-p_i)$ along its diagonal. Then an update can be written as $$\boldsymbol{\beta}^{t+1} = \boldsymbol{\beta}^{t} + (\mathbf{X}^T \mathbf{W} \mathbf{X})^{-1}\mathbf{X}^T(\mathbf{y}-\mathbf{p})$$ which after rearranging can be written as $$\boldsymbol{\beta}^{t+1}=(\mathbf{X}^T \mathbf{W} \mathbf{X})^{-1}\mathbf{X}^T\mathbf{W}(\mathbf{X}\boldsymbol{\beta}^{t}+\mathbf{W}^{-1}(\mathbf{y}-\mathbf{p}))$$. This is also known as the iteratively reweighted least squares algorithm (IRLS) because it has the form of a least squares update on a transformed response. A good starting value for the $\beta_j$ values is zero.

In [55]:
log_reg = LogisticRegression(penalty='none', solver='saga')
log_reg.fit(x, y)
log_reg.coef_

array([[0.4913098]])

In [63]:
X_ = np.concatenate([np.ones((N,1)), x], axis=1)
max_iter = 100
epsilon = 0.1
beta = np.zeros((p+1,))
for _ in range(max_iter):
    prob = sigmoid(np.dot(X_, beta))
    W = np.diag(prob*(1-prob))
    beta += inv(X_.T.dot(W).dot(X_)).dot(X_.T.dot(y-prob))
#     if np.sum(np.abs(old_beta - beta)) < epsilon:
#         break

In [64]:
beta

array([0.97427358, 0.56022729])

## Naive Bayes

Assume that $\mathrm{Pr}(x|G=j)=f_j(x)=\prod_{k=1}^{p} f_{jk}(x_k)$ that is that given the class the features are independent. This assumption is generally not true, but it simplifies estimation. Class conditional densities can be estimated via Gaussians or Kernel density estimate. For discrete features one can use a histogram. The prediction for a new example $x$ is then: $$\hat{y} = \mathrm{argmax}_j~\mathrm{Pr}(x|G=j)\mathrm{Pr}(G=j) = \mathrm{argmax}_j
~\prod_{k=1}^{p} f_{jk}(x) \frac{1}{N}\sum_i^N I(y_i = j).$$ Here $N$ is the number of training examples. Below we use Gaussian distributions for $f$. *prevalence* stands for $\mathrm{Pr}(G=j)$.

In [20]:
# [Mean(X|Y=0), Mean(X|Y=1)], shape=(p,2)
means = np.vstack([X[y==0].mean(0), X[y==1].mean(0)]).T
# [Var(X|Y=0), Var(X|Y=1)], shape=(p,2)
variances = np.vstack([X[y==0].var(0), X[y==1].var(0)]).T

In [22]:
def gaussian(x,m,v):
    return np.exp(-(x-m)**2/(2*v))/np.sqrt(2*np.pi*v)

In [36]:
# [P(Y=0), P(Y=1)], shape=(2,)
prevalence = np.array([1-y.mean(), y.mean()])

In [51]:
# Note: X.shape = (20,3), means.shape = (3,2)
# Add axis 2 to X to broadcast the last axis of means (that is classes)
# Add axis 0 to means and variances to broadcast of the zeroeth axis of X (that is data points)
def naive_bayes(X_):
    return np.argmax(gaussian(X_[:,:,None],means[None,:,:],variances[None,:,:]).prod(1) * prevalence,
                     axis=1)

# Multilayer Perceptron
Used from PyTorch example.

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# Randomly initialize weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2